In [1]:
import os
os.chdir('..')

In [120]:
import numpy as np
import pandas as pd

import ipfn
from modules.IPF import IPF

import matplotlib.pyplot as plt

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [62]:
df = pd.read_csv('../data/Chicago/full.csv')

In [6]:
df['total'] = np.ones(len(df))

In [7]:
df

,choice,travel_dow,trip_purpose,distance,hh_vehicles,hh_size,hh_bikes,hh_descr,hh_income,gender,age,license,education_level,work_status,departure_time,total
0,drive,7,HOME_OTHER,23.42579,2,2,0,2,6,1,66,1.0,6,FTE,9.333333,1.0
1,drive,7,OTHER,1.71259,2,2,0,2,6,1,66,1.0,6,FTE,12.083333,1.0
2,drive,7,HOME_OTHER,21.77887,2,2,0,2,6,1,66,1.0,6,FTE,15.500000,1.0
3,drive,7,SHOPPING,2.02603,2,2,0,2,6,1,66,1.0,6,FTE,17.500000,1.0
4,drive,7,SHOPPING,0.87691,2,2,0,2,6,1,66,1.0,6,FTE,18.250000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87941,drive,2,OTHER,2.42141,1,1,0,1,3,0,59,1.0,4,PTE,13.750000,1.0
87942,drive,2,HOME_OTHER,1.16283,1,1,0,1,3,0,59,1.0,4,PTE,14.083333,1.0
87943,drive,2,HOME_OTHER,0.54234,1,1,0,1,3,0,59,1.0,4,PTE,14.666667,1.0
87944,drive,2,OTHER,1.49183,1,1,0,1,3,0,59,1.0,4,PTE,15.333333,1.0


# IPF using pysynth

In [8]:
from typing import List, Optional

import pandas as pd

from modules.pysynth import ipf

SYNTHESIZERS = {
    'ipf': ipf.IPFSynthesizer,
}

DEFAULT_METHOD = 'ipf'

def synthesize(dataframe: pd.DataFrame,
               n_rows: Optional[int] = None,
               method: str = DEFAULT_METHOD,
               ignore_cols: List[str] = [],
               **kwargs) -> pd.DataFrame:
    '''Synthesize an analog to a given dataframe.
    Optional keyword arguments are passed to the selected synthesizer.
    :param dataframe: Data to be synthesized.
    :param n_rows: Number of output rows. If omitted, the same
        length as the input dataframe will be used.
    :param method: Method to use for synthesis. So far, only the `ipf` method
        using :class:`ipf.IPFSynthesizer` is available.
    :param ignore_cols: Columns not to be synthesized in the output (such as
        personal identifiers).
    '''
    synther = SYNTHESIZERS[method](ignore_cols=ignore_cols, **kwargs)
    synther.fit(dataframe)
    return synther.sample(n_rows)

In [9]:
synth_df = synthesize(df.sample(10), n_rows=None, method=DEFAULT_METHOD)


MemoryError: Unable to allocate 8.69 GiB for an array with shape (3, 4, 5, 10, 3, 5, 5, 2, 6, 2, 9, 1, 4, 3, 10, 1) and data type float64

In [ ]:
synth_df.head()

This shit doesn't work...

# IPFN

In [54]:
df = df[['choice', 'distance', 'license', 'total']]

In [56]:
aggr = []
dimensions = []

for c in df.columns:
    if c != 'total':
        aggr.append(df.groupby(c)['total'].sum())
        dimensions.append([c])
    
remaining = list(df.columns)
remaining.remove('total')
cols = list(df.columns)
cols.remove('total')
for c1 in cols:
    remaining.remove(c1)
    for c2 in remaining:
        if c1!=c2:
            aggr.append(df.groupby([c1,c2])['total'].sum())
            dimensions.append([c1,c2])

In [57]:
dimensions

[['choice'],
 ['distance'],
 ['license'],
 ['choice', 'distance'],
 ['choice', 'license'],
 ['distance', 'license']]

In [104]:
aggr

[choice
 cycle          845.0
 drive        52642.0
 passenger    19275.0
 pt            6019.0
 walk          9165.0
 Name: total, dtype: float64,
 distance
 0.00000     936.0
 0.00053      12.0
 0.00069       4.0
 0.00127       2.0
 0.00138       9.0
             ...  
 66.38100      2.0
 67.14308      2.0
 67.76025      6.0
 71.19393      4.0
 71.58191      2.0
 Name: total, Length: 44388, dtype: float64,
 license
 0.0    18185.0
 1.0    69761.0
 Name: total, dtype: float64,
 choice  distance
 cycle   0.00000     43.0
         0.00613      1.0
         0.01168      1.0
         0.01438      1.0
         0.03075      1.0
                     ... 
 walk    43.55082     1.0
         44.78233     1.0
         48.55131     4.0
         49.39073     1.0
         59.50744     1.0
 Name: total, Length: 53772, dtype: float64,
 choice     license
 cycle      0.0          257.0
            1.0          588.0
 drive      0.0            5.0
            1.0        52637.0
 passenger  0.0        1

In [58]:
IPF = ipfn.ipfn.ipfn(df, aggr, dimensions)

In [60]:
synth_df = IPF.iteration()

AttributeError: 'numpy.float64' object has no attribute 'loc'

Shit doesn't work either... Next one...

# IPF

https://github.com/juanshishido/IPF/blob/master/IPF.py

In [65]:
df = pd.read_csv('../data/Chicago/full_num.csv')

In [78]:
cols = np.array(df.sum(axis=0))

In [79]:
rows = np.array(df.sum(axis=1))

In [95]:
mat = IPF(rows, cols, random_fill=True)

4.4291811236983515e-12
4.0430833830753646e-12
4.037750921542009e-12
4.055156163164325e-12
4.044643991377198e-12


In [98]:
mat

array([[ 1.58491305, 10.4636413 , 12.90457473, ...,  6.19290716,
         3.12874889, 14.95276065],
       [ 4.35348441, 16.06122636, 12.3056249 , ..., 13.49160885,
         7.89595213,  5.7183025 ],
       [ 0.15642498, 10.62020412,  9.23265625, ...,  4.66856366,
         5.55509455, 26.06375685],
       ...,
       [ 3.07124769,  0.82586908,  3.03381091, ...,  2.05087178,
         5.49121998,  3.86304568],
       [ 1.2806624 ,  1.1521526 ,  4.17399903, ...,  4.39092333,
         1.23564923,  7.41741255],
       [ 0.85636832,  2.75008006,  9.67947492, ..., 11.09117458,
         4.98785221, 21.83004543]])

# ipf

https://github.com/shr264/IPF

In [105]:
df = pd.read_csv('../data/Chicago/full_num.csv')

In [106]:
cols = np.array(df.sum(axis=0))
rows = np.array(df.sum(axis=1))

In [112]:
from modules.ipf import ipf

In [113]:
ipf = ipf()

In [130]:
np.ones((len(rows), len(cols)))

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [146]:
mat2 = ipf.fit(initial_joint = np.ones((len(rows), len(cols))), marginal1 = rows, marginal2 = cols, maxitr = 10, method = 'mle')

Fitting using mle ...


In [147]:
synth_df = pd.DataFrame(mat2)

In [148]:
synth_df.columns = df.columns

In [149]:
synth_df

,choice,travel_dow,trip_purpose,distance,hh_vehicles,hh_size,hh_bikes,hh_descr,hh_income,gender,age,license,education_level,work_status,departure_time
0,2.373199,4.864140,5.132137,7.045914,2.668277,4.124688,2.150223,2.127599,7.407064,0.653346,62.274287,1.134658,5.399305,2.901468,19.502819
1,2.062955,4.228259,4.461221,6.124813,2.319458,3.585474,1.869128,1.849461,6.438750,0.567935,54.133271,0.986326,4.693463,2.522163,16.953247
2,2.455862,5.033567,5.310899,7.291336,2.761218,4.268358,2.225119,2.201707,7.665065,0.676103,64.443414,1.174180,5.587373,3.002531,20.182138
3,2.186043,4.480543,4.727405,6.490257,2.457851,3.799406,1.980652,1.959812,6.822926,0.601822,57.363198,1.045176,4.973504,2.672651,17.964784
4,2.178744,4.465582,4.711620,6.468585,2.449644,3.786719,1.974038,1.953267,6.800143,0.599812,57.171652,1.041686,4.956896,2.663727,17.904796
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87941,1.758904,3.605074,3.803701,5.222102,1.977603,3.057026,1.593645,1.576877,5.489770,0.484229,46.154797,0.840955,4.001713,2.150432,14.454580
87942,1.705404,3.495418,3.688004,5.063262,1.917450,2.964041,1.545171,1.528913,5.322788,0.469501,44.750906,0.815376,3.879993,2.085023,14.014915
87943,1.704724,3.494025,3.686534,5.061244,1.916686,2.962859,1.544556,1.528304,5.320667,0.469314,44.733074,0.815051,3.878447,2.084192,14.009330
87944,1.770861,3.629580,3.829558,5.257601,1.991046,3.077807,1.604478,1.587596,5.527088,0.487521,46.468548,0.846672,4.028916,2.165051,14.552839


In [111]:
!pip install statsmodels

     |████████████████████████████████| 9.5 MB 2.6 MB/s eta 0:00:01


In [150]:
df

,choice,travel_dow,trip_purpose,distance,hh_vehicles,hh_size,hh_bikes,hh_descr,hh_income,gender,age,license,education_level,work_status,departure_time
0,1,7,3,23.42579,2,2,0,2,6,1,66,1.0,6,0,9.333333
1,1,7,5,1.71259,2,2,0,2,6,1,66,1.0,6,0,12.083333
2,1,7,3,21.77887,2,2,0,2,6,1,66,1.0,6,0,15.500000
3,1,7,6,2.02603,2,2,0,2,6,1,66,1.0,6,0,17.500000
4,1,7,6,0.87691,2,2,0,2,6,1,66,1.0,6,0,18.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87941,1,2,5,2.42141,1,1,0,1,3,0,59,1.0,4,2,13.750000
87942,1,2,3,1.16283,1,1,0,1,3,0,59,1.0,4,2,14.083333
87943,1,2,3,0.54234,1,1,0,1,3,0,59,1.0,4,2,14.666667
87944,1,2,5,1.49183,1,1,0,1,3,0,59,1.0,4,2,15.333333
